In [17]:
import pandas as pd
import json
from typing import List, Dict
import re
from urllib.parse import urlparse, parse_qs, unquote
from datetime import datetime

class LogToDetailedConverter:
    """JSON 로그를 Excel 형태로 변환하는 간소화된 클래스"""
    def __init__(self):
        self.input_data = []
        self.detailed_data = []
        self.combined_data_no_comma = []

    def convert_log_files(self, input_log_path: str):
        """
        JSON 로그 파일을 2개의 Excel 파일로 변환
        - log_analysis_detailed.xlsx
        - log_analysis_combined_no_comma_filled.xlsx
        """
        try:
            # 1. 입력 로그 파일 읽기
            self._load_log_file(input_log_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 쉼표 제거된 결합 데이터로 변환
            self._convert_to_combined_no_comma()
            
            # 4. 누락된 행 채우기
            self._fill_missing_rows_in_detailed_data()
            self._fill_missing_rows_in_combined_no_comma()
            
            # 5. 파일 저장
            self._save_detailed_excel("./result/log_analysis_detailed.xlsx")
            self._save_combined_no_comma_excel("./result/log_analysis_combined_no_comma_filled.xlsx")
            
            print(f"✅ 변환 완료:")
            print(f"   - 상세 형태: ./result/log_analysis_detailed.xlsx")
            print(f"   - 결합 형태 (쉼표 제거): ./result/log_analysis_combined_no_comma_filled.xlsx")
            print(f"   - 처리된 로그: {len(self.input_data)}개")
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def _load_log_file(self, log_path: str):
        """JSON 로그 파일 로드 및 파싱 (오류 자동 수정 기능 포함)"""
        try:
            self.input_data = []
            json_errors = []
            fixed_count = 0
            
            with open(log_path, 'r', encoding='utf-8') as file:
                for line_no, line in enumerate(file, 1):
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        # JSON 파싱
                        log_entry = json.loads(line)
                        parsed_data = self._process_new_log_format(log_entry)
                        
                        if parsed_data:
                            parsed_data['log_line'] = line_no
                            self.input_data.append(parsed_data)
                    
                    except json.JSONDecodeError as e:
                        json_errors.append((line_no, str(e), line))
                        
                        # JSON 수정 시도
                        fixed_line = self._try_fix_json(line)
                        if fixed_line:
                            try:
                                log_entry = json.loads(fixed_line)
                                parsed_data = self._process_new_log_format(log_entry)
                                
                                if parsed_data:
                                    parsed_data['log_line'] = line_no
                                    self.input_data.append(parsed_data)
                                    fixed_count += 1
                                    print(f"✅ JSON 수정 성공 (라인 {line_no})")
                            except:
                                # 추가 수정 시도
                                fixed_line2 = self._force_fix_json(line)
                                if fixed_line2:
                                    try:
                                        log_entry = json.loads(fixed_line2)
                                        parsed_data = self._process_new_log_format(log_entry)
                                        
                                        if parsed_data:
                                            parsed_data['log_line'] = line_no
                                            self.input_data.append(parsed_data)
                                            fixed_count += 1
                                            print(f"✅ 강제 JSON 수정 성공 (라인 {line_no})")
                                    except:
                                        print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                                else:
                                    print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
                        else:
                            print(f"❌ JSON 수정 실패 (라인 {line_no}): {e}")
            
            # 오류 요약 출력
            if json_errors:
                print(f"⚠️ JSON 파싱 오류 발생: {len(json_errors)}개")
                if fixed_count > 0:
                    print(f"✅ 수정 성공: {fixed_count}개")
                print(f"✅ 최종 파싱된 로그: {len(self.input_data)}개")
            else:
                print(f"✅ 모든 JSON 파싱 성공")
            
            print(f"✅ 로그 파일 로드 완료: {len(self.input_data)}개 로그")
            
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {log_path}")
            raise
        except Exception as e:
            print(f"❌ 파일 로드 오류: {e}")
            raise

    def _process_new_log_format(self, log_entry: dict) -> dict:
        """새로운 로그 형식 처리: 직접 key-value 객체, 빈 값 제외, 특정 필드 제외"""
        # 제외할 필드들
        exclude_fields = {'client_ip', 'os_version', 'platform_type', 'device_model', 'device_maker', 'event_dt', 'nth_pcid', 'nth_sid', 'user_seq'}
        
        # 처리된 데이터
        processed_data = {}
        
        # event_dttm을 logtime으로 변환
        if 'event_dttm' in log_entry:
            processed_data['logtime'] = log_entry['event_dttm']
        
        # 각 key-value 쌍 처리
        for key, value in log_entry.items():
            # 제외 필드는 건너뛰기
            if key in exclude_fields:
                continue
            
            # event_dttm은 이미 logtime으로 처리했으므로 건너뛰기
            if key == 'event_dttm':
                continue
            
            # 빈 문자열("")인 값은 제외
            if value == "":
                continue
            
            # 유효한 key-value 쌍만 포함
            processed_data[key] = value
        
        return processed_data

    def _try_fix_json(self, line: str) -> str:
        """JSON 수정 시도 - 1차 수정"""
        try:
            # 1. 마지막 쉼표 제거
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # 2. 값 내부의 따옴표를 작은따옴표로 변경
            import re
            
            # 모든 "key":"value with quotes" 패턴을 찾아서 수정
            def fix_quotes_in_value(match):
                key = match.group(1)
                value = match.group(2)
                # 값 내부의 모든 따옴표를 작은따옴표로 변경
                fixed_value = value.replace('"', "'")
                return f'"{key}":"{fixed_value}"'
            
            # 값에 따옴표가 포함된 패턴 찾기
            problem_pattern = r'"([^"]+)":"([^"]*"[^"]*)"'
            
            # 여러 번 적용하여 모든 문제 해결
            prev_line = ""
            iterations = 0
            while line != prev_line and iterations < 5:
                prev_line = line
                line = re.sub(problem_pattern, fix_quotes_in_value, line)
                iterations += 1
            
            return line
            
        except Exception:
            return None

    def _force_fix_json(self, line: str) -> str:
        """JSON 강제 수정 - 2차 수정"""
        try:
            # 기본 정리
            if line.endswith(',}'):
                line = line[:-2] + '}'
            elif line.endswith(',]'):
                line = line[:-2] + ']'
            
            # JSON 구조 분석하여 키-값 쌍별로 처리
            if not (line.startswith('{') and line.endswith('}')):
                return None
            
            # 중괄호 제거
            content = line[1:-1]
            
            # 키-값 쌍들을 분리
            pairs = []
            current_pair = ""
            bracket_count = 0
            in_quotes = False
            escape_next = False
            
            for char in content:
                if escape_next:
                    current_pair += char
                    escape_next = False
                    continue
                
                if char == '\\':
                    current_pair += char
                    escape_next = True
                    continue
                
                if char == '"' and not escape_next:
                    in_quotes = not in_quotes
                
                if not in_quotes:
                    if char in '{}[]':
                        bracket_count += 1 if char in '{[' else -1
                    elif char == ',' and bracket_count == 0:
                        pairs.append(current_pair.strip())
                        current_pair = ""
                        continue
                
                current_pair += char
            
            if current_pair.strip():
                pairs.append(current_pair.strip())
            
            # 각 키-값 쌍을 수정
            fixed_pairs = []
            for pair in pairs:
                if ':' in pair:
                    try:
                        # 키와 값 분리 (첫 번째 콜론 기준)
                        colon_pos = pair.find(':')
                        key_part = pair[:colon_pos].strip()
                        value_part = pair[colon_pos+1:].strip()
                        
                        # 값 부분에서 따옴표로 둘러싸인 문자열인지 확인
                        if value_part.startswith('"') and value_part.endswith('"'):
                            # 값 내부의 따옴표를 작은따옴표로 변경
                            inner_value = value_part[1:-1]  # 양쪽 따옴표 제거
                            fixed_inner = inner_value.replace('"', "'")
                            fixed_value = f'"{fixed_inner}"'
                            fixed_pairs.append(f'{key_part}:{fixed_value}')
                        else:
                            fixed_pairs.append(pair)
                    except:
                        fixed_pairs.append(pair)
                else:
                    fixed_pairs.append(pair)
            
            # 다시 조합
            result = '{' + ','.join(fixed_pairs) + '}'
            return result
            
        except Exception:
            return None

    def _fix_korean_text(self, text: str) -> str:
        """깨진 한글을 올바르게 디코딩"""
        try:
            # URL 디코딩 먼저 시도
            from urllib.parse import unquote
            decoded = unquote(text, encoding='utf-8')
            
            # 깨진 한글 패턴이 있는지 확인
            if self._has_broken_korean(decoded):
                # Latin-1로 인코딩 후 UTF-8로 디코딩
                try:
                    fixed = decoded.encode('latin-1').decode('utf-8')
                    return fixed
                except:
                    pass
                    
                # 그래도 안되면 바이트 단위로 처리
                try:
                    byte_data = bytes([ord(c) for c in decoded if ord(c) < 256])
                    fixed = byte_data.decode('utf-8', errors='ignore')
                    return fixed
                except:
                    pass
                    
                # UTF-8 바이트 패턴으로 복구 시도
                try:
                    # 특정 깨진 패턴들을 직접 매핑
                    broken_patterns = {
                        'ê·¹ììëª¨': '극장용모',
                        'í¥ê¸°ë¡­ê²': '향기롭게',
                        'ì¤ê¸°ì¼ì´': '수기일이',
                        'ììëª¨': '용모',
                        'ê·¹': '극'
                    }
                    
                    result = decoded
                    for broken, fixed in broken_patterns.items():
                        result = result.replace(broken, fixed)
                    
                    if result != decoded:
                        return result
                        
                except:
                    pass
            
            return decoded
            
        except Exception:
            return text

    def _has_broken_korean(self, text: str) -> bool:
        """깨진 한글 패턴이 있는지 확인"""
        broken_patterns = [
            'ë¶', 'ìë', 'ê¼¬', 'ë', 'í¼', 'ì¸', 'ê¸°', 'ê¸', 'ìì¹', 'ì', 'ê',
            'ê·¹ì', 'í¥ê¸°', 'ì¤ê¸°', 'ììë', 'ëª¨'
        ]
        return any(pattern in text for pattern in broken_patterns)

    def _format_logtime(self, logtime_str: str) -> str:
        """LOGTIME을 yyyy-mm-dd HH:MM:SS 형식으로 변환"""
        try:
            if not logtime_str:
                return ''
            
            # 새로운 형식: "2025-06-05T15:04:00.041" (ISO 8601)
            if 'T' in logtime_str:
                try:
                    # ISO 8601 형식 파싱
                    if '.' in logtime_str:
                        dt = datetime.fromisoformat(logtime_str.replace('Z', '+00:00'))
                    else:
                        dt = datetime.fromisoformat(logtime_str.replace('Z', '+00:00'))
                    
                    # yyyy-mm-dd HH:MM:SS 형식으로 변환
                    formatted_time = dt.strftime('%Y-%m-%d %H:%M:%S')
                    return formatted_time
                except:
                    pass
            
            # 기존 형식: "04/Jun/2025:21:46:25 +0900"
            month_map = {
                'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
            }
            
            # 정규식으로 파싱
            pattern = r'(\d{2})/(\w{3})/(\d{4}):(\d{2}):(\d{2}):(\d{2})\s*\+\d{4}'
            match = re.match(pattern, logtime_str)
            
            if match:
                day, month_name, year, hour, minute, second = match.groups()
                month = month_map.get(month_name, '01')
                formatted_time = f"{year}-{month}-{day} {hour}:{minute}:{second}"
                return formatted_time
            else:
                return logtime_str
                
        except Exception as e:
            print(f"⚠️ LOGTIME 변환 오류: {e} - 원본: {logtime_str}")
            return logtime_str

    def _convert_to_detailed(self):
        """로그 데이터를 상세 key-value 행으로 변환"""
        self.detailed_data = []
        
        # 제외할 기본 필드들 (메타데이터)
        exclude_fields = {'log_line', 'logtime'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')
            
            # LOGTIME 포맷 변환
            raw_logtime = log_entry.get('logtime', '')
            formatted_logtime = self._format_logtime(raw_logtime)
            
            # 나머지 파라미터들을 key-value 쌍으로 처리
            for key, value in log_entry.items():
                if key in exclude_fields:
                    continue
                
                self.detailed_data.append({
                    'no': log_no,
                    'log_time': formatted_logtime,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'click_text': click_text,
                    'key': key,
                    'value': str(value) if value is not None else ''
                })
        
        print(f"✅ 상세 데이터 변환 완료: {len(self.detailed_data)}개 key-value 쌍")

    def _convert_to_combined_no_comma(self):
        """로그 데이터를 쉼표 제거된 결합 형태로 변환"""
        self.combined_data_no_comma = []
        
        # 제외할 기본 필드들
        exclude_fields = {'log_line', 'logtime', 'clientip', 'useragent', 'status'}
        
        for log_entry in self.input_data:
            log_no = log_entry.get('log_line', 0)
            
            # 기본 정보 추출
            page_id = log_entry.get('page_id', '')
            click_type = log_entry.get('click_type', '')
            act_type = log_entry.get('act_type', '')
            click_text = log_entry.get('click_text', '')
            
            # 나머지 파라미터들 수집
            keys = []
            values = []
            
            for key, value in log_entry.items():
                if key not in exclude_fields:
                    keys.append(key)
                    # 각 개별 value 내부의 쉼표만 제거
                    clean_value = str(value).replace(',', '') if value is not None else ''
                    values.append(clean_value)
            
            # 키와 값을 쉼표로 구분된 문자열로 결합
            keys_combined = ', '.join(keys) if keys else ''
            values_combined = ', '.join(values) if values else ''
            
            combined_entry = {
                'no': log_no,
                'page_id': page_id,
                'click_type': click_type,
                'act_type': act_type,
                'keys_combined': keys_combined,
                'values_combined': values_combined,
                'key_count': len(keys)
            }
            
            # click_text가 있으면 추가 (내부 쉼표 제거)
            if click_text:
                combined_entry['click_text'] = str(click_text).replace(',', '')
            
            self.combined_data_no_comma.append(combined_entry)
        
        print(f"✅ 쉼표 제거된 결합 데이터 변환 완료: {len(self.combined_data_no_comma)}개 로그")

    def _fill_missing_rows_in_detailed_data(self):
        """detailed_data에서 누락된 no 번호에 빈 행을 추가"""
        if not self.detailed_data:
            print("⚠️ 상세 데이터가 없어 누락된 행을 채울 수 없습니다.")
            return
        
        # 현재 no 값들 추출 (중복 제거)
        current_nos = list(set([item['no'] for item in self.detailed_data]))
        min_no = min(current_nos)
        max_no = max(current_nos)
        
        # 전체 범위의 no 리스트 생성
        full_range = set(range(min_no, max_no + 1))
        existing_nos = set(current_nos)
        missing_nos = sorted(full_range - existing_nos)
        
        if not missing_nos:
            print("✅ 상세 데이터에서 누락된 번호가 없습니다.")
            return
        
        print(f"📌 상세 데이터에서 누락된 번호: {missing_nos}")
        
        # 기본 빈 행 템플릿 생성
        template_row = self.detailed_data[0].copy()
        
        # 누락된 번호들에 대한 빈 행 생성
        for missing_no in missing_nos:
            empty_row = template_row.copy()
            empty_row['no'] = missing_no
            empty_row['log_time'] = ''
            empty_row['page_id'] = ''
            empty_row['click_type'] = ''
            empty_row['act_type'] = ''
            empty_row['click_text'] = ''
            empty_row['key'] = 'NO_LOG'
            empty_row['value'] = 'Missing Log Entry'
            
            self.detailed_data.append(empty_row)
        
        # no 순으로 정렬
        self.detailed_data.sort(key=lambda x: x['no'])
        
        print(f"✅ 상세 데이터에서 누락된 {len(missing_nos)}개 행을 추가했습니다.")

    def _fill_missing_rows_in_combined_no_comma(self):
        """combined_data_no_comma에서 누락된 no 번호에 빈 행을 추가"""
        if not self.combined_data_no_comma:
            print("⚠️ 쉼표 제거된 결합 데이터가 없어 누락된 행을 채울 수 없습니다.")
            return
        
        # 현재 no 값들 추출
        current_nos = [item['no'] for item in self.combined_data_no_comma]
        min_no = min(current_nos)
        max_no = max(current_nos)
        
        # 전체 범위의 no 리스트 생성
        full_range = set(range(min_no, max_no + 1))
        existing_nos = set(current_nos)
        missing_nos = sorted(full_range - existing_nos)
        
        if not missing_nos:
            print("✅ 쉼표 제거된 결합 데이터에서 누락된 번호가 없습니다.")
            return
        
        print(f"📌 쉼표 제거된 결합 데이터에서 누락된 번호: {missing_nos}")
        
        # 기본 빈 행 템플릿 생성
        template_row = self.combined_data_no_comma[0].copy()
        
        # 누락된 번호들에 대한 빈 행 생성
        for missing_no in missing_nos:
            empty_row = template_row.copy()
            empty_row['no'] = missing_no
            empty_row['page_id'] = ''
            empty_row['click_type'] = ''
            empty_row['act_type'] = ''
            empty_row['keys_combined'] = ''
            empty_row['values_combined'] = ''
            empty_row['key_count'] = 0
            
            # click_text가 있으면 빈 값으로 설정
            if 'click_text' in empty_row:
                empty_row['click_text'] = ''
            
            self.combined_data_no_comma.append(empty_row)
        
        # no 순으로 정렬
        self.combined_data_no_comma.sort(key=lambda x: x['no'])
        
        print(f"✅ 쉼표 제거된 결합 데이터에서 누락된 {len(missing_nos)}개 행을 추가했습니다.")

    def _save_detailed_excel(self, output_path: str):
        """상세 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_data)
            
            if df.empty:
                print("⚠️ 저장할 상세 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의
            column_order = ['no', 'log_time', 'page_id', 'act_type', 'click_type', 'click_text', 'key', 'value']
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 Excel 파일 저장: {output_path}")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def _save_combined_no_comma_excel(self, output_path: str):
        """value 내부 쉼표 제거된 결합 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.combined_data_no_comma)
            
            if df.empty:
                print("⚠️ 저장할 결합 데이터가 없습니다.")
                return
            
            # 컬럼 순서 정의
            column_order = ['no', 'page_id', 'click_type', 'act_type', 'keys_combined', 'values_combined', 'key_count']
            
            # click_text 컬럼이 있으면 추가
            if 'click_text' in df.columns:
                column_order.insert(4, 'click_text')
            
            # 존재하는 컬럼만 선택
            available_columns = [col for col in column_order if col in df.columns]
            df = df[available_columns]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 결합 Excel 파일 저장: {output_path}")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise


def main():
    """메인 실행 함수"""
    converter = LogToDetailedConverter()
    
    print("=== JSON 로그를 Excel 형태로 변환 ===")
    print("📌 생성 파일:")
    print("   - log_analysis_detailed.xlsx (상세 형태)")
    print("   - log_analysis_combined_no_comma_filled.xlsx (결합 형태, 쉼표 제거)\n")
    
    # 로그 파일 변환
    converter.convert_log_files("./log/log_file.txt")


if __name__ == "__main__":
    main()

=== JSON 로그를 Excel 형태로 변환 ===
📌 생성 파일:
   - log_analysis_detailed.xlsx (상세 형태)
   - log_analysis_combined_no_comma_filled.xlsx (결합 형태, 쉼표 제거)

✅ 모든 JSON 파싱 성공
✅ 로그 파일 로드 완료: 3개 로그
✅ 상세 데이터 변환 완료: 16개 key-value 쌍
✅ 쉼표 제거된 결합 데이터 변환 완료: 3개 로그
✅ 상세 데이터에서 누락된 번호가 없습니다.
✅ 쉼표 제거된 결합 데이터에서 누락된 번호가 없습니다.
✅ 상세 Excel 파일 저장: ./result/log_analysis_detailed.xlsx
✅ 결합 Excel 파일 저장: ./result/log_analysis_combined_no_comma_filled.xlsx
✅ 변환 완료:
   - 상세 형태: ./result/log_analysis_detailed.xlsx
   - 결합 형태 (쉼표 제거): ./result/log_analysis_combined_no_comma_filled.xlsx
   - 처리된 로그: 3개
